In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


**Image Segmentation**

In [ ]:
img = cv2.imread("")

In [ ]:
# Convert the input image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
# blur the image to denoise
blurred_image = skimage.filters.gaussian(gray, sigma=1.0)

fig, ax = plt.subplots()
plt.imshow(blurred_image, cmap="gray")

In [ ]:
# create a histogram of the blurred grayscale image
histogram, bin_edges = np.histogram(blurred_image, bins=256, range=(0.0, 1.0))

fig, ax = plt.subplots()
plt.plot(bin_edges[0:-1], histogram)
plt.title("Grayscale Histogram")
plt.xlabel("grayscale value")
plt.ylabel("pixels")
plt.xlim(0, 1.0)

In [ ]:
# create a mask based on the threshold
t = 0.4
binary_mask = blurred_image > t

fig, ax = plt.subplots()
plt.imshow(binary_mask, cmap="gray")

In [ ]:
# apply thresholding on gray image
ret,thresh = cv2.threshold(gray,160,255,0)

fig, ax = plt.subplots()
plt.imshow(thresh, cmap="gray")

Make a Histogram of RGB colors 0-255 each

In [1]:

# image = skimage.io.imread(fname="/content/sharktooth.jpg", as_gray=True)

# # display the image
# fig, ax = plt.subplots()
# plt.imshow(image, cmap="gray")

Calculate Area of teeth size

# **Moment invariant**

Reading image file

In [4]:
img1 = cv2.imread("Segmented_Teeth/segmented tooth1_1.png")
img2 = cv2.imread("Segmented_Teeth/segmented tooth1_2.png")

In [5]:
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

In [9]:
moments1 = cv2.HuMoments(cv2.moments(img1)).flatten()
print("ORIGINAL MOMENTS for img 1: {}".format(moments1))

moments2 = cv2.HuMoments(cv2.moments(img2)).flatten()
print("ORIGINAL MOMENTS for img 2: {}".format(moments2))


ORIGINAL MOMENTS for img 1: [9.31673872e-04 3.94367501e-07 1.16589393e-10 1.50416801e-11
 6.29560476e-22 9.44115823e-15 2.07860839e-23]
ORIGINAL MOMENTS for img 2: [9.32487012e-04 4.19574583e-07 7.85269033e-11 1.14095140e-11
 3.37913841e-22 7.05015323e-15 4.94653700e-23]


In [17]:
# for i in range(0,7):
#    moments1[i] = -np.sign(1.0, moments1[i])*np.math.log10(np.abs(moments1[i]))
   
#-1* copysign(1.0, huMoments[i]) * log10(abs(huMoments[i]))

lmoment1 = np.abs(np.log10(moments1))
print(lmoment1)

[ -3.03073608  -6.40409888  -9.93334096 -10.82270365 -21.20096254
 -14.02497472 -22.68222732]


In [8]:
#distance between same type of teeth fossil with different angle
d1 = cv2.matchShapes(img1,img2,cv2.CONTOURS_MATCH_I1,0)
d2 = cv2.matchShapes(img1,img2,cv2.CONTOURS_MATCH_I2,0) 
d3 = cv2.matchShapes(img1,img2,cv2.CONTOURS_MATCH_I3,0)

print(d1)
print(d2)
print(d3)

4.125290634915402e-05
0.00037887547626658247
0.0001250110421262778


In [10]:
# compute the Hu Moments feature vector for the entire image and show it
moments = cv2.HuMoments(cv2.moments(img)).flatten()
print("ORIGINAL MOMENTS: {}".format(moments))
